In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from itertools import combinations
from tqdm import tqdm
from time import time

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, KFold


In [17]:
X_train = pd.read_csv('X_train.csv', index_col=0).sort_index()
y_train = pd.read_csv('y_train.csv', index_col=0).sort_index()
X_test = pd.read_csv('X_test.csv', index_col=0).sort_index()

In [19]:
print(f"Total number of individuals : {len(X_train)}.")
years = X_train['Year'].unique()
for year in years:
  count_year = len(X_train[X_train['Year']==year])
  print(f"Pour l'année {year}, il y a {count_year} individus.")

print(f"Total number of individuals in test: {len(X_test)}.")

Total number of individuals : 1172086.
Pour l'année 2022, il y a 412647 individus.
Pour l'année 2018, il y a 411320 individus.
Pour l'année 2015, il y a 348119 individus.
Total number of individuals in test: 586044.


# Choosing which columns to keep

In [ ]:
X_train.shape

In [ ]:
count_null = X_train.isna().sum()
kept_cols = list(count_null[count_null < 0.5 * len(X_train)].index)

exclude_substrings = ['_total_timing', '_average_score', 'CNTRYID', 'CNTSTUID', 'BOOKID']

final_cols = [col for col in kept_cols if not any(substr in col for substr in exclude_substrings)]

X_train = X_train[final_cols]
X_test = X_test[final_cols]

In [ ]:
#The different types of columns 
categorical1 = ['Year', 'CNT', 'CYC', 'STRATUM', 'OECD', 'ADMINMODE', 'ST004D01T', 'IMMIG', 'CNTSCHID']
categorical_cols = []
binary_cols = []
numeric_cols = []
categorical_threshold = 5

for col in X_train.columns:
    if len(set(X_train[col].dropna().unique()))==2:
        binary_cols.append(col)
    elif col in categorical1:
        categorical_cols.append(col)
    elif X_train[col].dtype == 'object':
        categorical_cols.append(col)
    elif pd.api.types.is_numeric_dtype(X_train[col]):
        numeric_cols.append(col)

print(f"Categorical columns: {len(categorical_cols)}")
print(f"Binary columns (0/1): {len(binary_cols)}")
print(f"Real/numeric columns: {len(numeric_cols)}")


In [ ]:
for col in categorical_cols:
    print(f"For {col}: {len(X_train[col].unique())} unique values and {len(X_train[X_train[col].isna()])} missing values.")

# Imputing missing data

In [ ]:
X_train.isna().sum()

In [ ]:
# Numerical columns
imputer_num = SimpleImputer(strategy='median')
X_train[numeric_cols] = imputer_num.fit_transform(X_train[numeric_cols])  # fit on train
X_test[numeric_cols] = imputer_num.transform(X_test[numeric_cols])

# Categorical or binary columns
imputer_cat = SimpleImputer(strategy='most_frequent')
X_train[binary_cols] = imputer_cat.fit_transform(X_train[binary_cols])
X_test[binary_cols] = imputer_cat.transform(X_test[binary_cols])

X_train[categorical_cols] = imputer_cat.fit_transform(X_train[categorical_cols])
X_test[categorical_cols] = imputer_cat.transform(X_test[categorical_cols])


In [ ]:
X_train.isna().sum()

# Scaling and One hot encoding

In [ ]:
# Standard Scaling of the numerical variable 
do_standard_scaler = True 

if do_standard_scaler:
    scaler = StandardScaler()
    scaler.fit(X_train[numeric_cols])
    X_train_final = scaler.transform(X_train[numeric_cols])
    X_train_final = pd.DataFrame(X_train_final, columns=numeric_cols, index=X_train.index)

    X_test_final = scaler.transform(X_test[numeric_cols])
    X_test_final = pd.DataFrame(X_test_final, columns=numeric_cols, index=X_test.index)
else:
    X_train_final = X_train[numeric_cols].copy()
    X_test_final = X_test[numeric_cols].copy()

# One hot encoding of the binary
for cat_variable in binary_cols:
    dummies = pd.get_dummies(X_train[cat_variable], prefix=cat_variable)
    X_train_final = pd.concat([X_train_final, dummies], axis=1)

    dummies = pd.get_dummies(X_test[cat_variable], prefix=cat_variable)
    for col in [c for c in X_train_final.columns if c.startswith(cat_variable)]:
        if col not in dummies.columns:
            dummies[col] = 0
    # Ensure same column order as training set
    dummies = dummies[[c for c in X_train_final.columns if c.startswith(cat_variable)]]
    X_test_final = pd.concat([X_test_final, dummies], axis=1)

#One hot encoding of some of the categorical
for cat_variable in ['Year', 'CNT', 'CYC', 'OECD', 'IMMIG']:
    dummies = pd.get_dummies(X_train[cat_variable], prefix=cat_variable)
    X_train_final = pd.concat([X_train_final, dummies], axis=1)
    if cat_variable in X_train_final.columns:
        X_train_final = X_train_final.drop(columns=[cat_variable])

    dummies = pd.get_dummies(X_test[cat_variable], prefix=cat_variable)
    # Add any missing columns from training set with 0s
    for col in [c for c in X_train_final.columns if c.startswith(cat_variable)]:
        if col not in dummies.columns:
            dummies[col] = 0
    # Ensure same column order as training set
    dummies = dummies[[c for c in X_train_final.columns if c.startswith(cat_variable)]]
    X_test_final = pd.concat([X_test_final, dummies], axis=1)

X_test = X_test_final.copy()


In [ ]:
X_train_final, X_test_final, y_train, y_test = train_test_split(X_train_final, y_train, test_size=0.7, random_state=42)


# Remove the highly correlated features

In [ ]:
corr_matrix = X_train_final.corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

high_corr = corr_matrix.where(mask).stack().sort_values(ascending=False)
high_corr = high_corr[high_corr > 0.9]

to_drop = set()
for f1, f2 in high_corr.index:
    if f2 not in to_drop:
        to_drop.add(f2)

print(f"Dropping {len(to_drop)} variables.")

X_train_final = X_train_final.drop(columns=list(to_drop))
X_test_final = X_test_final.drop(columns=list(to_drop))
X_test = X_test.drop(columns=list(to_drop))

## Regressions 

In [33]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.01),
    "Gradient Boosting 1": XGBRegressor(tree_method="hist", max_depth=6, n_estimators=200),
    #"Gradient Boosting 2": GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=3, subsample=0.7, min_samples_leaf=5),
    "Random Forest": RandomForestRegressor( n_estimators=300, max_depth=20, min_samples_leaf=5,max_features='sqrt'),
    "Elastic Net": ElasticNet(alpha=0.01, l1_ratio=0.5),
    "Decision Tree": DecisionTreeRegressor(max_depth=None),
    "Support Vector Regressor": SVR(kernel="rbf", C=1.0, epsilon=0.1)
}


In [ ]:
results = {}
non_na_cols = X_train_final.columns[X_train_final.notna().all()].tolist()

for name, model in models.items():
    beginning = time()
    print(name)
    X_fit = X_train_final[non_na_cols]
    print("X shape:", X_fit.shape, "y shape:", y_train.shape)
    if name in ["Linear Regression", "Ridge Regression", "Lasso Regression"]:
        model.fit(X_fit, y_train)
    else: 
        model.fit(X_fit, y_train.values.ravel())
        #print("X shape:", X_train.shape, "y shape:", y_train.shape)
        #model.fit(X_train_final, y_train)
        #y_pred = model.predict(X_train)
    X_train_fit = X_train_final[non_na_cols]
    X_test_fit  = X_test_final[non_na_cols]

    y_pred_train = model.predict(X_train_fit)
    y_pred_test = model.predict(X_test_fit)

    # Train metrics
    r2_train = r2_score(y_train, y_pred_train)

    # Test metrics
    r2_test = r2_score(y_test, y_pred_test)

    print("Train R2:", r2_train)
    print("Test  R2:", r2_test)

    results[name] = {"Train R2": r2_train,
                    "Test R2": r2_test}
    print(time() - beginning)



Linear Regression
X shape: (351625, 274) y shape: (351625, 1)
Train R2: 0.07625233862552083 Train MSE: 13771.149358131059
Test  R2: 0.0744769099326652 Test  MSE: 13822.827023774838
7.1768410205841064
Ridge Regression
X shape: (351625, 274) y shape: (351625, 1)
Train R2: 0.07624072915465663 Train MSE: 13771.322431107506
Test  R2: 0.07447528579201534 Test  MSE: 13822.85128056055
4.315684080123901
Lasso Regression
X shape: (351625, 274) y shape: (351625, 1)
Train R2: 0.0760142678018243 Train MSE: 13774.698497152489
Test  R2: 0.07433731776612262 Test  MSE: 13824.911853848447
32.15350580215454
Gradient Boosting 1
X shape: (351625, 274) y shape: (351625, 1)
Train R2: 0.19140517711639404 Train MSE: 12054.460625640997
Test  R2: 0.07571858167648315 Test  MSE: 13804.280167256717
7.858360767364502
Random Forest
X shape: (351625, 274) y shape: (351625, 1)
Train R2: 0.2076305714648583 Train MSE: 11812.574151406514
Test  R2: 0.08091350666942831 Test  MSE: 13726.695480144093
209.3693928718567
Elastic

In [ ]:
y_train.describe()

# Cross validation 

In [ ]:
models_params = {
    "Linear Regression": (LinearRegression(), {}),
    "Ridge Regression": (Ridge(), {"alpha": [0.1, 1.0, 10]}),
    "Lasso Regression": (Lasso(), {"alpha": [0.001, 0.01, 0.1]}),
    "Elastic Net": (ElasticNet(), {"alpha": [0.001,0.01,0.1,1],"l1_ratio":[0.1,0.5,0.9]}),
    "Random Forest": (
        RandomForestRegressor(),
        {
            "n_estimators":[200,300,500],
            "max_depth":[10,20,30,None],
            "min_samples_split":[2,5,10],
            "min_samples_leaf":[1,2,5],
            "max_features":["sqrt","log2",0.5]
        }
    ),
    "XGBoost": (
        XGBRegressor(tree_method="hist"),
        {
            "n_estimators":[200,400,600],
            "max_depth":[3,5,7,10],
            "learning_rate":[0.01,0.05,0.1],
            "subsample":[0.6,0.8,1.0],
            "colsample_bytree":[0.6,0.8,1.0],
            "gamma":[0,1,5]
        }
    ),
    "Decision Tree": (
        DecisionTreeRegressor(),
        {"max_depth":[None,10,20,30], "min_samples_split":[2,5,10], "min_samples_leaf":[1,2,5]}
    ),
    "SVR": (
        SVR(),
        {"C":[0.1,1,10], "epsilon":[0.01,0.1,0.5], "gamma":["scale","auto"]}
    )
}

non_na_cols = X_train_final.columns[X_train_final.notna().all()].tolist()
X_train_fit = X_train_final[non_na_cols]
X_test_fit  = X_test_final[non_na_cols]

results = {}
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# 3️⃣ Loop over models
for name, (model, params) in models_params.items():
    print(f"\n{name}")
    start = time()
    
    if params:  # if hyperparameters exist, do RandomizedSearchCV
        search = RandomizedSearchCV(
            estimator=model,
            param_distributions=params,
            n_iter=15,
            cv=cv,
            scoring="neg_mean_squared_error",
            n_jobs=-1,
            verbose=0,
            random_state=42
        )
        search.fit(X_train_fit, y_train.values.ravel())
        best_model = search.best_estimator_
        print("Best params:", search.best_params_)
    else:
        best_model = model
        best_model.fit(X_train_fit, y_train)

    # Predictions
    y_pred_train = best_model.predict(X_train_fit)
    y_pred_test  = best_model.predict(X_test_fit)

    # Metrics
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train  = r2_score(y_train, y_pred_train)
    mse_test  = mean_squared_error(y_test, y_pred_test)
    r2_test   = r2_score(y_test, y_pred_test)

    results[name] = {
        "Train MSE": mse_train,
        "Train R2": r2_train,
        "Test MSE": mse_test,
        "Test R2": r2_test
    }

    print(f"Train R2: {r2_train:.4f}, Test R2: {r2_test:.4f}, Time: {time()-start:.1f}s")




